La indicación 

    %%script false --no-raise-error

Implica que esa celda de código no va a ser ejecutada.

Estas celdas suponen un tiempo de 15h. 

En vez de ejecutarlas cada vez que quiero recomendar, leo un CSV con los embeddings procesados en una ejecución anterior.

In [1]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors #Importar modelo desde fichero
from gensim.models import Word2Vec #Importar word2vec
from datetime import datetime #tracking de tiempo en realizar de embeddings
from scipy import spatial #similitud del coseno
from operator import itemgetter # comparación de tuplas

Cargamos nuestro dataset de películas, y por si acaso (no debería haber ninguna), eliminamos las películas con descripciones nulas.

In [2]:
df = pd.read_csv("movieDatasetClean.csv")
df= df[df['description'].notnull()]

Realizamos un corpus de palabras que contenga todas las reviews de nuestro dataset separadas en palabras.

También obtenemos el modelo básico de word2vec "GoogleNews-vectors-negative300". Este modelo fue entrenado por Google usando un dataset de Google News con alrededor de 100 mil millones de palabras.

Este modelo contiene vectores 300-dimensonales para 3 millones de palabras.

In [3]:
%%script false --no-raise-error
corpus = []
for words in df['description']:
    corpus.append(words.split())
    
print("Comienzo: ",datetime.now())
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
print("Cargado: ",datetime.now())

Como hemos explicado antes, word2vec devuelve un vector dado una palabra.

Por lo tanto, para interpretar las decripciones de las películas podemos realizar la media entre todos los vectores "palabra" de cada descripción y ver si los resultados son prometedores.

In [4]:
%%script false --no-raise-error
def get_mean_vector(model, words):
    words = [word for word in words if word in list(model.index_to_key)]
    if len(words) >= 1:
        return np.mean(model[words], axis=0)
    else:
        return []

Esta es la celda de código más pesada, donde obtendríamos los vectores medios de cada descripción.

In [5]:
%%script false --no-raise-error
i=0
vectors=[]
for doc in corpus:
    vec = get_mean_vector(model, doc)
    if len(vec) > 0:
        vectors.append(vec)
        if i%500==0:
            print(i,datetime.now())
        i=i+1
    else:
        vectors.append([])

vectors_full=pd.DataFrame(vectors)
vectors_movie=df.join(vectors_full)

Llegados a este punto podemos importar los vectores que habíamos calculado en una ejecución anterior y empezar a ver los resultados.

In [6]:
#vectors_movie.to_csv("movies_word2vec")
movie_ds=pd.read_csv("movies_word2vec.csv")

vectors_movie=movie_ds.iloc[: , 14:314]

Como medida de similitud entre descripciones de películas (vectores), voy a usar la distancia del coseno.

La distancia del coseno se suele utilizar como media de similitud en procesamiento de texto.

La similitud del coseno tiene en cuenta la orientación de los vectores y no la magnitud.

In [7]:
def get_film_similarity(id1,id2):
    return 1 - spatial.distance.cosine(vectors_movie.iloc[id1],vectors_movie.iloc[id2] )

Habiendo descrito nuestra medida de distancia podemos, dada una película, buscar las películas que tienen descripciones más parecidas.

In [8]:
def recomend5(id_movie):
    toplist=[]
    for i in range (0,14209):
        if (i==id_movie):
            continue
        similarity=get_film_similarity(i,id_movie)
        toplist.append(tuple((similarity, i)))

    toplist.sort(key=itemgetter(0),reverse=True)
    print("Dada la pelicula ",movie_ds.iloc[id_movie]["title"])
    i=1
    for peli in toplist[0:5]:
        print("Recomendación", i,"(",int(peli[0]*100),"%):",movie_ds.iloc[peli[1]]["title"],"(genero:",movie_ds.iloc[peli[1]]["genre"],"nota:",movie_ds.iloc[peli[1]]["rating"],")")
        i+=1

Habiendo visto todo el código, vamos a comprobar las recomendaciones de varias películas

In [9]:
recomend5(1) #dado el padrino

Dada la pelicula  The Godfather
Recomendación 1 ( 85 %): The Godfather: Part III (genero:  Action nota: 7.7 )
Recomendación 2 ( 85 %): This So-Called Disaster: Sam Shepard Directs the Late Henry Moss (genero:  Documentary nota: tbd )
Recomendación 3 ( 84 %): The Godfather: Part II (genero:  Drama nota: 9.0 )
Recomendación 4 ( 84 %): Hello Lonesome (genero:  Drama nota: tbd )
Recomendación 5 ( 83 %): Gonzo: The Life and Work of Dr. Hunter S. Thompson (genero:  Biography nota: 7.7 )


In [10]:
recomend5(1878) #dado los vengadores

Dada la pelicula  Avengers: Endgame
Recomendación 1 ( 80 %): Captain Marvel (genero:  Action nota: 2.9 )
Recomendación 2 ( 79 %): Avengers: Age of Ultron (genero:  Action nota: 7.0 )
Recomendación 3 ( 79 %): Avengers: Infinity War (genero:  Action nota: 8.6 )
Recomendación 4 ( 78 %): Harry Potter and the Deathly Hallows: Part I (genero:  Action nota: 7.6 )
Recomendación 5 ( 78 %): Captain America: Civil War (genero:  Action nota: 8.2 )


In [11]:
recomend5(267) #ambas star wars

Dada la pelicula  Star Wars: Episode IV - A New Hope
Recomendación 1 ( 85 %): I Am Not a Serial Killer (genero:  Thriller nota: 7.0 )
Recomendación 2 ( 84 %): The Myth of Fingerprints (genero:  Drama nota: tbd )
Recomendación 3 ( 84 %): Yves Saint Laurent (genero:  Biography nota: 5.7 )
Recomendación 4 ( 84 %): Star Wars: Episode V - The Empire Strikes Back (genero:  Action nota: 9.0 )
Recomendación 5 ( 82 %): Blame (genero:  Drama nota: 7.8 )


In [12]:
recomend5(444) 

Dada la pelicula  Goldfinger
Recomendación 1 ( 69 %): What If (genero:  Drama nota: 7.2 )
Recomendación 2 ( 61 %): On Her Majesty's Secret Service (genero:  Action nota: 7.4 )
Recomendación 3 ( 61 %): Moonraker (genero:  Action nota: 5.8 )
Recomendación 4 ( 60 %): The Dictator (genero:  Comedy nota: 6.4 )
Recomendación 5 ( 58 %): Stonehearst Asylum (genero:  Thriller nota: 6.8 )


In [13]:
recomend5(630) 

Dada la pelicula  Harry Potter and the Deathly Hallows: Part 2
Recomendación 1 ( 87 %): Hellboy II: The Golden Army (genero:  Action nota: 7.4 )
Recomendación 2 ( 86 %): Aquaman (genero:  Action nota: 6.9 )
Recomendación 3 ( 86 %): Avengers: Age of Ultron (genero:  Action nota: 7.0 )
Recomendación 4 ( 86 %): The Hudsucker Proxy (genero:  Drama nota: 7.2 )
Recomendación 5 ( 86 %): Blizzard of Souls (genero:  Drama nota: tbd )


In [14]:
recomend5(7700) 

Dada la pelicula  Jumanji: Welcome to the Jungle
Recomendación 1 ( 77 %): 101 Dalmatians (genero:  Adventure nota: 6.0 )
Recomendación 2 ( 77 %): Marcello Mastroianni: I Remember (genero:  Biography nota: 5.5 )
Recomendación 3 ( 77 %): Working Girls (genero:  Drama nota: tbd )
Recomendación 4 ( 76 %): Thinner (genero:  Fantasy nota: 8.1 )
Recomendación 5 ( 76 %): Cameraperson (genero:  Documentary nota: 7.0 )
